In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm

In [ ]:
train_data = pd.read_csv("../train.csv")
test_data = pd.read_csv("../test.csv")

train_data.head()

In [ ]:
print(f"Train Shape: {train_data.shape}")
print(f"Test Shape: {test_data.shape}")

In [ ]:
X_train, y_train = train_data['x'][:10000], train_data['y'][:10000]
X_test, y_test = test_data['x'][:2000], test_data['y'][:2000]

print(f"Train data shape: {X_train.shape} | {y_train.shape}")
print(f"Test data shape: {X_test.shape} | {y_test.shape}")